### Enexis Kleinverbruikgegevens dataset
https://www.enexis.nl/over-ons/wat-bieden-we/andere-diensten/open-data 


Beschrijving:

https://www.enexis.nl/over-ons/-/media/documenten/open-data/toelichting-kleinverbruiksgegevens-open-data-enexis-netbeheer.pdf?modified=00010101000000&la=nl-nl&hash=1A66620D8DBA0D55A55767B507C535BC218B0B14

We hadden al de csv files per jaar vanaf 2010 maar er is nu ook een nieuwe csv file voor 2021 beschikbaar. op de website van Enexis staan alleen de laatste 3 jaar.

Belangrijkste attributen in de dataset:
- POSTCODE_VAN en POSTCODE_TOT, PC6
- PRODUCTSOORT, zowel Gas als Elektriciteit. Filter toepassen op alleen Elektriciteit
- AANSLUITINGEN_AANTAL, aantal aansluitingen in het postcode gebied
- LEVERINGSRICHTING_PERC, Percentage van de aansluitingen dat netto elektriciteits- of gasverbruik heeft. Dit percentage wordt lager naarmate er meer teruglevering plaatsvindt (b.v. vanwege zonnepanelen) 
- SOORT_AANSLUITING  de meest voorkomende ‘Soort aansluiting’, Opties voor elektriciteit: (”#zekeringen x # ampère”) 1x25, 1x35, 1x50, 3x25, 3x35, 3x50, 3x63, 3x80
- SJV_GEMIDDELD, Het gemiddeld standaardjaarverbruik (SJV) in kWh. Het verwachte jaarverbruik van een afnemer op een netaansluiting bij gestandaardiseerde condities en op basis van een genormaliseerd jaar. Bij teruglevering dan is het SJV gesaldeerd.

In [166]:
import pandas as pd
import numpy as np
import os
import glob
import zipfile
import seaborn as sns

In [169]:
# show all columns in the dataframe
pd.set_option('max_columns', None)

In [172]:
os.getcwd()

'c:\\Users\\ericr\\OneDrive - Actondata\\Projecten\\jads_enexis\\notebooks'

In [175]:
# variables used in script
# path to Data folder
DATA = '../data/raw/Enexis_kleinverbruikgegevens'

# Ga naar de folder waarin de data staat
if 'Enexis_kleinverbruikgegevens' not in os.getcwd():
    os.chdir(DATA)

In [178]:
df = pd.DataFrame()
for j in glob.glob('Enexis_kleinverbruiksgegevens*.zip'):
    print(j)
    appenddata = pd.read_csv(j, sep=';', thousands='.', decimal=',',  encoding= 'unicode_escape')
    # jaar kolom toevoegen op basis van de file name
    appenddata['JAAR'] = j[-8:-4] # Het standaardjaarverbruik is het verwachte jaarverbruik gebaseerd op vorige jaar
    df = df.append(appenddata,ignore_index=True)

Enexis_kleinverbruiksgegevens_01012010.zip
Enexis_kleinverbruiksgegevens_01012011.zip
Enexis_kleinverbruiksgegevens_01012012.zip
Enexis_kleinverbruiksgegevens_01012013.zip
Enexis_kleinverbruiksgegevens_01012014.zip
Enexis_kleinverbruiksgegevens_01012015.zip
Enexis_kleinverbruiksgegevens_01012016.zip
Enexis_kleinverbruiksgegevens_01012017.zip
Enexis_kleinverbruiksgegevens_01012018.zip
Enexis_kleinverbruiksgegevens_01012019.zip
Enexis_kleinverbruiksgegevens_01012020.zip
Enexis_kleinverbruiksgegevens_01012021.zip


In [181]:
df.columns

Index(['NETBEHEERDER', 'NETGEBIED', 'STRAATNAAM', 'POSTCODE_VAN',
       'POSTCODE_TOT', 'WOONPLAATS', 'LANDCODE', 'PRODUCTSOORT',
       'VERBRUIKSSEGMENT', 'LEVERINGSRICHTING_PERC', 'AANSLUITINGEN_AANTAL',
       'FYSIEKE_STATUS_PERC', 'SOORT_AANSLUITING_PERC', 'SOORT_AANSLUITING',
       'SJV_GEMIDDELD', 'SJV_LAAG_TARIEF_PERC', 'SLIMME_METER_PERC', 'JAAR'],
      dtype='object')

In [184]:
#df["SOORT_AANSLUITING"].value_counts(dropna=False)
df = df.astype({'JAAR':'int64'})
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2563201 entries, 0 to 2563200
Data columns (total 18 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   NETBEHEERDER            object 
 1   NETGEBIED               object 
 2   STRAATNAAM              object 
 3   POSTCODE_VAN            object 
 4   POSTCODE_TOT            object 
 5   WOONPLAATS              object 
 6   LANDCODE                object 
 7   PRODUCTSOORT            object 
 8   VERBRUIKSSEGMENT        object 
 9   LEVERINGSRICHTING_PERC  float64
 10  AANSLUITINGEN_AANTAL    int64  
 11  FYSIEKE_STATUS_PERC     int64  
 12  SOORT_AANSLUITING_PERC  float64
 13  SOORT_AANSLUITING       object 
 14  SJV_GEMIDDELD           float64
 15  SJV_LAAG_TARIEF_PERC    float64
 16  SLIMME_METER_PERC       float64
 17  JAAR                    int64  
dtypes: float64(5), int64(3), object(10)
memory usage: 352.0+ MB


In [187]:
# filter op elektriciteit
df = df[df['PRODUCTSOORT'] == 'ELK']

In [190]:
# check missing values
missing_total = df.isna().sum().sort_values(ascending=False)
print('Number of attributes with Null values: ' + str(len(missing_total[missing_total > 0])))
print(missing_total[missing_total > 0])

Number of attributes with Null values: 3
SOORT_AANSLUITING         107581
SOORT_AANSLUITING_PERC    107512
LEVERINGSRICHTING_PERC       958
dtype: int64


In [193]:
# fill missing values 
df['SOORT_AANSLUITING'] = df['SOORT_AANSLUITING'].fillna('Onbekend')
df['SOORT_AANSLUITING_PERC'] = df['SOORT_AANSLUITING_PERC'].fillna(0)
df['LEVERINGSRICHTING_PERC'] = df['LEVERINGSRICHTING_PERC'].fillna(0)

In [196]:
df.head()

,NETBEHEERDER,NETGEBIED,STRAATNAAM,POSTCODE_VAN,POSTCODE_TOT,WOONPLAATS,LANDCODE,PRODUCTSOORT,VERBRUIKSSEGMENT,LEVERINGSRICHTING_PERC,AANSLUITINGEN_AANTAL,FYSIEKE_STATUS_PERC,SOORT_AANSLUITING_PERC,SOORT_AANSLUITING,SJV_GEMIDDELD,SJV_LAAG_TARIEF_PERC,SLIMME_METER_PERC,JAAR
0,Enexis B.V.,ENEXIS,Sasdijk,4251AB,4251AB,WERKENDAM,NL,ELK,KVB,100.0,16,100,0.0,Onbekend,4282.0,25.0,0.0,2010
1,Enexis B.V.,ENEXIS,Sasdijk,4251AC,4251AC,WERKENDAM,NL,ELK,KVB,100.0,11,100,0.0,Onbekend,5113.0,10.0,0.0,2010
2,Enexis B.V.,ENEXIS,Sasdijk,4251AD,4251AD,WERKENDAM,NL,ELK,KVB,100.0,30,100,0.0,Onbekend,4809.0,34.0,0.0,2010
3,Enexis B.V.,ENEXIS,Nieuweweg,4251AE,4251AG,WERKENDAM,NL,ELK,KVB,100.0,21,100,0.0,Onbekend,5015.0,44.0,0.0,2010
4,Enexis B.V.,ENEXIS,Koppenhof,4251AH,4251AH,WERKENDAM,NL,ELK,KVB,100.0,12,100,0.0,Onbekend,3074.0,22.0,0.0,2010


In [199]:
df[df["JAAR"] > 2010]

,NETBEHEERDER,NETGEBIED,STRAATNAAM,POSTCODE_VAN,POSTCODE_TOT,WOONPLAATS,LANDCODE,PRODUCTSOORT,VERBRUIKSSEGMENT,LEVERINGSRICHTING_PERC,AANSLUITINGEN_AANTAL,FYSIEKE_STATUS_PERC,SOORT_AANSLUITING_PERC,SOORT_AANSLUITING,SJV_GEMIDDELD,SJV_LAAG_TARIEF_PERC,SLIMME_METER_PERC,JAAR
190478,Enexis B.V.,ENEXIS,Sasdijk,4251AB,4251AB,WERKENDAM,NL,ELK,KVB,100.00,16,100,50.0,1X25,4961.00,34.00,0.00,2011
190479,Enexis B.V.,ENEXIS,Sasdijk,4251AC,4251AC,WERKENDAM,NL,ELK,KVB,100.00,11,100,64.0,1X25,4517.00,13.00,0.00,2011
190480,Enexis B.V.,ENEXIS,Sasdijk,4251AD,4251AD,WERKENDAM,NL,ELK,KVB,100.00,30,100,43.0,3X25,3675.00,31.00,0.00,2011
190481,Enexis B.V.,ENEXIS,Nieuweweg,4251AE,4251AG,WERKENDAM,NL,ELK,KVB,100.00,21,100,52.0,1X25,5249.00,48.00,0.00,2011
190482,Enexis B.V.,ENEXIS,Koppenhof,4251AH,4251AH,WERKENDAM,NL,ELK,KVB,100.00,12,100,33.0,1X25,3226.00,33.00,0.00,2011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2563191,Enexis B.V.,ENEXIS,Jacob Tilbusscherweg,9998 XB,9998 XB,ROTTUM GN,NL,ELK,KVB,66.67,21,100,61.9,3x25,7390.48,46.48,85.71,2021
2563193,Enexis B.V.,ENEXIS,Jan Boerweg,9998 XC,9998 XD,ROTTUM GN,NL,ELK,KVB,55.00,20,100,55.0,3x25,2440.35,41.43,55.00,2021
2563195,Enexis B.V.,ENEXIS,Knolweg,9998 XE,9999 XG,ROTTUM GN,NL,ELK,KVB,8.33,12,100,50.0,3x25,11102.83,52.30,100.00,2021
2563197,Enexis B.V.,ENEXIS,Stitswerderweg,9999 XH,9999 XJ,STITSWERD,NL,ELK,KVB,50.00,16,100,62.5,3x25,6420.81,38.98,75.00,2021


In [202]:
# Poging 1. Veeeeerrrrry slow maar werkt wel. Om dit sneller te krijgen moeten we apply of np.vectorize gebruiken
# print ("Aansluitsoort bijwerken:",end="")
# i = 0
# for postcode in postcode_van:
#     # Print een punt elke 1000 postcodes om de voortgang te kunnen zien
#     if not i%1000:
#         print (".", end="")
#     i = i + 1

#     # haal het soort aansluiting op uit 2011. Filter geeft een 1x1 dataframe terug! (als er iets gevonden is). Vandaar de list conversie
#     soort_aansluiting = postcode_aansluiting[postcode]

#     # als er geen aansluiting is gevonden, ga verder met de volgende postcode
#     if not soort_aansluiting:
#         continue

#     # Selecteer het eerstgevonden jaar de aanname is dat de aansluiting niet veranderd is
#     soort_aansluiting = soort_aansluiting[0]
#     # DEBUG: print (f"{postcode} heeft aansluiting {soort_aansluiting}")

#     # Vervang Onbekend door de soort aansluiting die is gevonden in een later jaar voor dezelfde postcode_van
#     df.loc[(df.POSTCODE_VAN == postcode) & (df.JAAR == 2010) & (df.SOORT_AANSLUITING == "Onbekend"), "SOORT_AANSLUITING"] = soort_aansluiting

In [205]:
# Maak test set met random onbekende aansluitingen in 2010
df_unknown = df[df.SOORT_AANSLUITING == "Onbekend"]
count = df_unknown.shape[0]
print (f"Er zijn {count} onbekende aansluitingen")

# Maak een sample set met 10 rijen met onbekende aansluiting
random_index_list = df_unknown.sample(n=10).index.values
print (f"Lijst met random indices met onbekende aansluiting = {random_index_list}")

# check aanname dat de onbekende aansluitingen alleen in 2010 zitten
df_unknown = df[(df.SOORT_AANSLUITING == "Onbekend") & ((df.JAAR == 2010) | (df.JAAR == 2017))]
re_count = df_unknown.shape[0]
print (f"Er zijn {count} onbekende aansluitingen in 2010 en 2017")
assert(count == re_count)

# cross check tegen dataframe
print ("Geselecteerde rijen:")
df.loc[random_index_list]

Er zijn 107581 onbekende aansluitingen
Lijst met random indices met onbekende aansluiting = [ 25804 100110 131402 182100 118849 177825  62987 113018 101891  17308]
Er zijn 107581 onbekende aansluitingen in 2010 en 2017
Geselecteerde rijen:


,NETBEHEERDER,NETGEBIED,STRAATNAAM,POSTCODE_VAN,POSTCODE_TOT,WOONPLAATS,LANDCODE,PRODUCTSOORT,VERBRUIKSSEGMENT,LEVERINGSRICHTING_PERC,AANSLUITINGEN_AANTAL,FYSIEKE_STATUS_PERC,SOORT_AANSLUITING_PERC,SOORT_AANSLUITING,SJV_GEMIDDELD,SJV_LAAG_TARIEF_PERC,SLIMME_METER_PERC,JAAR
25804,Enexis B.V.,ENEXIS,van Hogendorpplein,5051SV,5051SW,GOIRLE,NL,ELK,KVB,100.0,20,100,0.0,Onbekend,6865.0,31.0,0.0,2010
100110,Enexis B.V.,ENEXIS,Jonker van Weerststraat,6441SW,6441SW,BRUNSSUM,NL,ELK,KVB,100.0,31,100,0.0,Onbekend,4558.0,49.0,0.0,2010
131402,Enexis B.V.,ENEXIS,Zevenster,7918AV,7918TA,NIEUWLANDE,NL,ELK,KVB,100.0,29,100,0.0,Onbekend,4223.0,33.0,0.0,2010
182100,Enexis B.V.,ENEXIS,Voermanhaven,9742VR,9742VR,GRONINGEN,NL,ELK,KVB,100.0,28,100,0.0,Onbekend,3414.0,56.0,0.0,2010
118849,Enexis B.V.,ENEXIS,Vikkerhoekweg,7555PL,7555PL,HENGELO OV,NL,ELK,KVB,100.0,19,100,0.0,Onbekend,5186.0,51.0,0.0,2010
177825,Enexis B.V.,ENEXIS,Belcamposingel,9721ME,9721ME,GRONINGEN,NL,ELK,KVB,100.0,15,100,0.0,Onbekend,3797.0,1.0,0.0,2010
62987,Enexis B.V.,ENEXIS,Steenstraat,5831JH,5831JH,BOXMEER,NL,ELK,KVB,100.0,17,100,0.0,Onbekend,24337.0,59.0,0.0,2010
113018,Enexis B.V.,ENEXIS,Veldkampstraat,7513ZB,7513ZE,ENSCHEDE,NL,ELK,KVB,100.0,34,100,0.0,Onbekend,4019.0,48.0,0.0,2010
101891,Enexis B.V.,ENEXIS,Schiltgen,6462GJ,6462GJ,KERKRADE,NL,ELK,KVB,100.0,19,100,0.0,Onbekend,3900.0,43.0,0.0,2010
17308,Enexis B.V.,ENEXIS,Leeuwerikstraat,4881WZ,4881WZ,ZUNDERT,NL,ELK,KVB,100.0,50,100,0.0,Onbekend,6077.0,30.0,0.0,2010


In [208]:
import time

df = df.astype({'SOORT_AANSLUITING':'str'})

start = time.time()
# Onbekende aansluitingsoort in 2010 gaan we proberen aan te vullen door de postcode op te zoeken in latere jaren
row_count = df[df.SOORT_AANSLUITING == "Onbekend"].shape[0]
print (f"Aantal onbekende aansluitingen = {row_count}")

# Voeg postcode toe met soort aansluiting aan de dictionary die we later gebruiken om de onbekende aansluitingen bij te werken
postcode_aansluiting = {}
def maak_dictionary(postcode, aansluiting):
    global postcode_aansluiting
    if not postcode in postcode_aansluiting:
        postcode_aansluiting[postcode] = aansluiting

# Verzamel eerst alle data die we hebben per postcode en soort aansluiting in een dictionary
df_partial = df[df.JAAR > 2010]
np.vectorize(maak_dictionary)(df_partial.POSTCODE_VAN, df_partial.SOORT_AANSLUITING)

print (f"Aansluitingen per postcode verzameld. Aantal postcodes = {len(postcode_aansluiting)}")

# Poging 2. Gebruik vectorization. 
print ("Aansluitsoort bijwerken voor 2010 en 2017")
def werk_soort_aansluiting_bij(postcode):
    if postcode in postcode_aansluiting:
        return postcode_aansluiting[postcode]
    return "Onbekend"
df.loc[(df.JAAR == 2010) | (df.JAAR == 2017), 'SOORT_AANSLUITING'] = np.vectorize(werk_soort_aansluiting_bij)(df[(df.JAAR == 2010) | (df.JAAR == 2017)].POSTCODE_VAN)

# wat hebben overgehouden?
row_count = df[df.SOORT_AANSLUITING == "Onbekend"].shape[0]
print (f"Aantal onbekende aansluitingen na aanvulling = {row_count}")

end = time.time()
print (f"Totale executie tijd: {end - start} seconden")

Aantal onbekende aansluitingen = 137
Aansluitingen per postcode verzameld. Aantal postcodes = 243014
Aansluitsoort bijwerken voor 2010 en 2017
Aantal onbekende aansluitingen na aanvulling = 137
Totale executie tijd: 0.8247146606445312 seconden


In [211]:
# Zijn alle onbekende aansluitingen verdwenen? (Het kan zijn dat er een enkele aansluiting onbekend is gebleven. Daar hebben we dan geen data van)
df.loc[random_index_list]

,NETBEHEERDER,NETGEBIED,STRAATNAAM,POSTCODE_VAN,POSTCODE_TOT,WOONPLAATS,LANDCODE,PRODUCTSOORT,VERBRUIKSSEGMENT,LEVERINGSRICHTING_PERC,AANSLUITINGEN_AANTAL,FYSIEKE_STATUS_PERC,SOORT_AANSLUITING_PERC,SOORT_AANSLUITING,SJV_GEMIDDELD,SJV_LAAG_TARIEF_PERC,SLIMME_METER_PERC,JAAR
25804,Enexis B.V.,ENEXIS,van Hogendorpplein,5051SV,5051SW,GOIRLE,NL,ELK,KVB,100.0,20,100,0.0,1X25,6865.0,31.0,0.0,2010
100110,Enexis B.V.,ENEXIS,Jonker van Weerststraat,6441SW,6441SW,BRUNSSUM,NL,ELK,KVB,100.0,31,100,0.0,1X25,4558.0,49.0,0.0,2010
131402,Enexis B.V.,ENEXIS,Zevenster,7918AV,7918TA,NIEUWLANDE,NL,ELK,KVB,100.0,29,100,0.0,1X35,4223.0,33.0,0.0,2010
182100,Enexis B.V.,ENEXIS,Voermanhaven,9742VR,9742VR,GRONINGEN,NL,ELK,KVB,100.0,28,100,0.0,1X40,3414.0,56.0,0.0,2010
118849,Enexis B.V.,ENEXIS,Vikkerhoekweg,7555PL,7555PL,HENGELO OV,NL,ELK,KVB,100.0,19,100,0.0,1X35,5186.0,51.0,0.0,2010
177825,Enexis B.V.,ENEXIS,Belcamposingel,9721ME,9721ME,GRONINGEN,NL,ELK,KVB,100.0,15,100,0.0,3X25,3797.0,1.0,0.0,2010
62987,Enexis B.V.,ENEXIS,Steenstraat,5831JH,5831JH,BOXMEER,NL,ELK,KVB,100.0,17,100,0.0,3X25,24337.0,59.0,0.0,2010
113018,Enexis B.V.,ENEXIS,Veldkampstraat,7513ZB,7513ZE,ENSCHEDE,NL,ELK,KVB,100.0,34,100,0.0,1X35,4019.0,48.0,0.0,2010
101891,Enexis B.V.,ENEXIS,Schiltgen,6462GJ,6462GJ,KERKRADE,NL,ELK,KVB,100.0,19,100,0.0,1X25,3900.0,43.0,0.0,2010
17308,Enexis B.V.,ENEXIS,Leeuwerikstraat,4881WZ,4881WZ,ZUNDERT,NL,ELK,KVB,100.0,50,100,0.0,3X25,6077.0,30.0,0.0,2010


In [214]:
# Overgebleven rijen in 2010 zonder aansluiting
df[df['SOORT_AANSLUITING'] == "Onbekend"].head()

,NETBEHEERDER,NETGEBIED,STRAATNAAM,POSTCODE_VAN,POSTCODE_TOT,WOONPLAATS,LANDCODE,PRODUCTSOORT,VERBRUIKSSEGMENT,LEVERINGSRICHTING_PERC,AANSLUITINGEN_AANTAL,FYSIEKE_STATUS_PERC,SOORT_AANSLUITING_PERC,SOORT_AANSLUITING,SJV_GEMIDDELD,SJV_LAAG_TARIEF_PERC,SLIMME_METER_PERC,JAAR
60,Enexis B.V.,ENEXIS,Laantje,4251EL,4251EL,WERKENDAM,NL,ELK,KVB,100.0,11,100,0.0,Onbekend,7279.0,25.0,0.0,2010
631,Enexis B.V.,ENEXIS,Oude Kerkstraat,4271BB,4271BB,DUSSEN,NL,ELK,KVB,100.0,13,100,0.0,Onbekend,4827.0,33.0,0.0,2010
851,Enexis B.V.,ENEXIS,Hugo de Grootstraat,4285AL,4285AL,WOUDRICHEM,NL,ELK,KVB,100.0,14,100,0.0,Onbekend,3399.0,44.0,0.0,2010
2058,Enexis B.V.,ENEXIS,Markiezaatsweg,4616PC,4617AA,BERGEN OP ZOOM,NL,ELK,KVB,100.0,27,100,0.0,Onbekend,4143.0,3.0,0.0,2010
2272,Enexis B.V.,ENEXIS,Speenkruidstraat,4621AW,4621AZ,BERGEN OP ZOOM,NL,ELK,KVB,100.0,31,100,0.0,Onbekend,1506.0,8.0,0.0,2010


In [217]:
# totale SJV voor alle onbekende aansluitingen
som_onbekend = df.loc[df.SOORT_AANSLUITING == "Onbekend", "SJV_GEMIDDELD"].sum()
print (som_onbekend)

881852.0


In [220]:
# totale SJV voor 2010
som_totaal = df[df.JAAR == 2010].SJV_GEMIDDELD.sum()
print (som_totaal)

526581294.0


In [223]:
# percentage gemist
percentage = format(som_onbekend / som_totaal * 100, '.2f')
print (percentage + "%")

0.17%


In [226]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1391527 entries, 0 to 2563199
Data columns (total 18 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   NETBEHEERDER            1391527 non-null  object 
 1   NETGEBIED               1391527 non-null  object 
 2   STRAATNAAM              1391527 non-null  object 
 3   POSTCODE_VAN            1391527 non-null  object 
 4   POSTCODE_TOT            1391527 non-null  object 
 5   WOONPLAATS              1391527 non-null  object 
 6   LANDCODE                1391527 non-null  object 
 7   PRODUCTSOORT            1391527 non-null  object 
 8   VERBRUIKSSEGMENT        1391527 non-null  object 
 9   LEVERINGSRICHTING_PERC  1391527 non-null  float64
 10  AANSLUITINGEN_AANTAL    1391527 non-null  int64  
 11  FYSIEKE_STATUS_PERC     1391527 non-null  int64  
 12  SOORT_AANSLUITING_PERC  1391527 non-null  float64
 13  SOORT_AANSLUITING       1391527 non-null  object 
 14  SJ

In [229]:
# Verwijder de rijen met onbekende aansluiting. Ze maken niet echt veel uit op het geheel
df = df[df.SOORT_AANSLUITING != "Onbekend"]

In [232]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1391390 entries, 0 to 2563199
Data columns (total 18 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   NETBEHEERDER            1391390 non-null  object 
 1   NETGEBIED               1391390 non-null  object 
 2   STRAATNAAM              1391390 non-null  object 
 3   POSTCODE_VAN            1391390 non-null  object 
 4   POSTCODE_TOT            1391390 non-null  object 
 5   WOONPLAATS              1391390 non-null  object 
 6   LANDCODE                1391390 non-null  object 
 7   PRODUCTSOORT            1391390 non-null  object 
 8   VERBRUIKSSEGMENT        1391390 non-null  object 
 9   LEVERINGSRICHTING_PERC  1391390 non-null  float64
 10  AANSLUITINGEN_AANTAL    1391390 non-null  int64  
 11  FYSIEKE_STATUS_PERC     1391390 non-null  int64  
 12  SOORT_AANSLUITING_PERC  1391390 non-null  float64
 13  SOORT_AANSLUITING       1391390 non-null  object 
 14  SJ

In [235]:
# voeg attibuut toe met numerieke deel van de postcode
df['PC4'] = df['POSTCODE_VAN'].str[0:4]

In [238]:
# extra attribuut toevoegen voor totale standaard jaarverbruik
df['SJV_TOTAAL'] = round(df['AANSLUITINGEN_AANTAL'] * df['SJV_GEMIDDELD'] / 1000) # delen door 1000 om MW ipv KW te krijgen

In [241]:
os.getcwd()

'c:\\Users\\ericr\\OneDrive - Actondata\\Projecten\\jads_enexis\\data\\raw\\Enexis_kleinverbruikgegevens'

In [244]:
# data locatie om de ht5 file op te slaan
data_processed_location = '../../processed'

if 'processed' not in os.getcwd():
    os.chdir(data_processed_location)

In [247]:
try:
    os.remove('kleinverbruikgegevens_data.h5')
except:
    print ('File nog in gebruik of niet gevonden')

store = pd.HDFStore('kleinverbruikgegevens_data.h5')
store['kleinverbruikgegevens_data'] = df
store.close()

In [250]:
# group by pc4
df_pc4 = df.groupby(['PC4','JAAR']
                    , as_index=False
                    ).agg({'SJV_TOTAAL': 'sum'
                           ,'AANSLUITINGEN_AANTAL': 'sum'
                           ,'LEVERINGSRICHTING_PERC': 'mean'
                           }
                         )

In [253]:
df_pc4.head()

,PC4,JAAR,SJV_TOTAAL,AANSLUITINGEN_AANTAL,LEVERINGSRICHTING_PERC
0,4251,2010,21802.0,4325,99.953704
1,4251,2011,23084.0,4413,99.953917
2,4251,2012,24006.0,4486,99.854545
3,4251,2013,23728.0,4512,99.660633
4,4251,2014,23623.0,4544,99.094595


In [256]:
NOTEBOOKS = "../../notebooks"

if 'notebooks' not in os.getcwd():
    os.chdir(NOTEBOOKS)